In [1]:
import argparse
import os
import datetime as dt
from glob import glob
import numpy as np
from os import path
from shutil import copyfile
from subprocess import run
import xarray
import yaml
from boundary import flood_missing, Segment

def read_config(config_file):
    with open(config_file, 'r') as stream:
        config = yaml.safe_load(stream)
    return config

def merge_segment_files(output_dir, source):
    """Merge separate segment files into one file per source of BGC data.
    I used to do this with CDO, but CDO overwrites the attributes for time
    and I can't figure out how to disable that.
    Instead, this appends one file at a time with NCO.
    """
    infiles = glob(path.join(output_dir, f'bgc_{source}_0*'))
    if len(infiles) <= 1:
        print(f'Not merging {source}; found {len(infiles)} files.')
        return None
    outfile = path.join(output_dir, f'bgc_{source}.nc')
    for i, f in enumerate(infiles):
        if i == 0:
            # Copy the first file as the final output file
            copyfile(f, outfile)
        else:
            # Append subsequent files to the copy of the first file
            run([f'ncks -A {f} {outfile}'], shell=True)



In [2]:
    # parser = argparse.ArgumentParser(description='Generate BGC tracers obc.')
    # parser.add_argument('--config', dest='config_file', default='bgc_obc.yaml', help='Path to the YAML configuration file')
    # args = parser.parse_args()
    config_file = 'bgc_obc.yaml'
    config = read_config(config_file)

    output_dir = config['output_dir']
    grid_file = config['grid_file']

    # Create output directory if it doesn't exist
    if not path.exists(output_dir):
        os.makedirs(output_dir)

    # Regional model domain and boundaries
    hgrid = xarray.open_dataset(grid_file)

    # Load segments
    segments = []
    for seg_config in config.get('segments', []):
        segment = Segment(seg_config['id'], seg_config['border'], hgrid, output_dir=output_dir)
        segments.append(segment)

    time0 = dt.datetime.strptime(str(config['time0']), '%Y-%m-%d')
    last_time = dt.datetime.strptime(str(config['last_time']), '%Y-%m-%d')  

    # WOA for no3, o2, po4, sio4
    woa_file = config['woa_file'] 

    # ESPER for dic and alk 
    esper_file = config['esper_file'] 

    # COBALT climatology for remaining variables
    cobalt_file = config['cobalt_file'] 


In [3]:
woa_climo = xarray.open_dataset(woa_file)
woa_climo.load()  # avoids some errors?
woa_climo = woa_climo.rename(depth='z')

esper = (
    xarray.open_dataset(esper_file)
    .rename({'st_ocean': 'z', 'yt_ocean': 'lat', 'xt_ocean': 'lon'})
    .rename({'Alk': 'alk', 'DIC': 'dic'})
    * 1e-6  # micromoles -> moles
)

initial = esper.isel(time=0)
initial['time'] = [time0]
last = esper.isel(time=-1)
last['time'] = [dt.datetime(2022, 1, 1)]
esper = xarray.concat([initial, esper, last], dim='time')
esper = esper.transpose('time', 'z', 'lat', 'lon')



cobalt_vars = [
    # 'alk',
    'cadet_arag',
    'cadet_calc',
    # 'dic',
#     'dic14',
#     'do14',
#     'do14c', 
#     'di14c',
    'fed', 
    'fedi',
    'felg',
    'fedet',
    'fesm',
    'ldon',
    'ldop', 
    'lith', 
    'lithdet', 
    'nbact', 
    'ndet', 
    'ndi', 
    'nlg', 
    'nsm', 
#     'nh3', 
    'nh4', 
    # 'no3', 
    # 'o2', 
    'pdet', 
    # 'po4', 
    'srdon', 
    'srdop', 
    'sldon', 
    'sldop', 
    'sidet', 
    'silg', 
    # 'sio4', 
    'nsmz', 
    'nmdz', 
    'nlgz'
]
cobalt = (
    xarray.open_mfdataset(cobalt_file)
    .rename({'st_ocean': 'z', 'geolat_t': 'lat', 'geolon_t': 'lon'})
    [cobalt_vars]
)
cobalt['time'] = [time0]

# Flood cobalt all together here, in part so that the lat and lon coordinates can be re-added.
cobalt_flooded = xarray.merge((
    flood_missing(cobalt[v], xdim='xt_ocean', ydim='yt_ocean', zdim='z') for v in cobalt.data_vars
))
# Need to load or else xesmf will fail when trying to recognize coordinates.
cobalt_flooded = cobalt_flooded.load()    
cobalt_flooded = cobalt_flooded.assign_coords(lat=cobalt['lat'], lon=cobalt['lon'])

# For 4P, create medium properties from large
for v in ['si', 'fe', 'n']:
    cobalt_flooded[f'{v}md'] = cobalt_flooded[f'{v}lg']

# For variable n:p, create p from n
cobalt_flooded['psm'] = cobalt_flooded['nsm'] / 24.0
cobalt_flooded['pmd'] = cobalt_flooded['nmd'] / 20.0
cobalt_flooded['plg'] = cobalt_flooded['nlg'] / 14.0
cobalt_flooded['pdi'] = cobalt_flooded['ndi'] / 40.0

common_kws = dict(write=False)

In [4]:
woa_climo.time

<xarray.DataArray 'time' (time: 12)>
array([cftime.DatetimeNoLeap(1, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 2, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 3, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 4, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 5, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 6, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 7, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 8, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 9, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 10, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 11, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1, 12, 1, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 0001-01-01 00:00:00 ... 0001-12-01 00:00:00

In [8]:
# def teste(i):
scale_factor = 1e-6
maxref = 1e4
for i,seg in enumerate(segments):
    print(i, 'WOA')
    aux = (seg.regrid_tracer(woa_climo[v]*scale_factor, regrid_suffix='woa_bgc', flood=True, periodic=False, **common_kws) for v in woa_climo)
    woa_seg = xarray.merge(aux)


    #extrapolating data using nearest horizontal point
    for varbs in woa_seg.data_vars.keys():
        # removing max values (in this case, woa has 
        woa_seg[varbs] = woa_seg[varbs].where(woa_seg[varbs].values<maxref,other=np.nan)
        for coords in sorted(list(woa_seg[varbs].coords.keys())):  # horizontal coords MUST be extrapolated first!!
            if 'time' in coords:
                continue
            woa_seg[varbs] = woa_seg[varbs].ffill(dim=coords)
            woa_seg[varbs] = woa_seg[varbs].bfill(dim=coords)
    # this configuration makes mom understand this is a climatology
    # woa_seg['time'].attrs['units'] = 'days since 0001-01-01'
    # woa_seg['time'].attrs['calendar'] = 'noleap'
    woa_seg['time'].attrs['modulo'] = ' '
    woa_seg['time'].attrs['cartesian_axis'] = 'T'
    # woa_seg = woa_seg.drop_duplicates('time')
    seg.to_netcdf(woa_seg, 'bgc_woa')

    # # ESPER
    print(i, 'ESPER')
    # No flooding due to size
    esper_seg = xarray.merge((seg.regrid_tracer(esper[v], regrid_suffix='esper', flood=False, periodic=False, **common_kws) for v in esper))
    # Make sure no negative values were produced, just in case.
    for v in esper_seg.data_vars:
        esper_seg[v] = np.clip(esper_seg[v], 0.0, None)
    esper_seg = seg.add_coords(esper_seg)
    esper_seg = esper_seg.drop_duplicates('time')
    seg.to_netcdf(esper_seg, 'bgc_esper')


    print(i, 'COBALT')
    # COBALT
    cobalt_seg = xarray.merge(
        (seg.regrid_tracer(cobalt_flooded[v], regrid_suffix='cobalt', flood=False, periodic=True, **common_kws) for v in cobalt_flooded)
    )
    # Make sure no negative values were produced, just in case.
    for v in cobalt_seg.data_vars:
        cobalt_seg[v] = np.clip(cobalt_seg[v], 0.0, None)
    cobalt_seg = cobalt_seg.drop_duplicates('time')
    cobalt_seg = seg.add_coords(cobalt_seg)
    
    for v in woa_seg.data_vars:
        woa_seg[v] = np.clip(woa_seg[v], 0.0, None)
    
    seg.to_netcdf(cobalt_seg, 'bgc_cobalt')


0 WOA
0 ESPER
0 COBALT
1 WOA
1 ESPER
1 COBALT


In [9]:
for source in ['woa', 'esper', 'cobalt']:
    merge_segment_files(output_dir, source)


In [7]:
output_dir# ls ../NWA0.25.COBALT/INPUT

'../data/output'